In [24]:
import easyocr
from PIL import Image

# Crie um leitor (reader)
reader = easyocr.Reader(['pt'])  # Especifica o idioma

# Ler o texto da imagem
results = reader.readtext('6.jpg')

# Imprimir os resultados
for (bbox, text, prob) in results:
    print(f'Texto: {text}, Confiança: {prob}')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: DlsEOVERs, Confiança: 0.2485222514511449
Texto: sL, Confiança: 0.13545443916121866
Texto: ProgFoD, Confiança: 0.2931641876061566


In [17]:
import cv2
import easyocr

# Carregar a imagem em escala de cinza
image = cv2.imread('5.jpg', cv2.IMREAD_GRAYSCALE)

image_contraste = cv2.equalizeHist(image)

# Aplicar Unsharp Masking
gaussian_blur = cv2.GaussianBlur(image_contraste, (9, 9), 10.0)
sharpened = cv2.addWeighted(image_contraste, 1.5, gaussian_blur, -0.5, 0)

# Salvar a imagem ajustada
cv2.imwrite('imagem_nitida.jpg', sharpened)

# Ler a imagem ajustada
reader = easyocr.Reader(['pt'])
results = reader.readtext(sharpened, text_threshold=0.7, low_text=0.4, link_threshold=0.4)

# Imprimir os resultados
for (bbox, text, prob) in results:
    print(f'Texto: {text}, Confiança: {prob}')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: @YF5J92, Confiança: 0.8342566675176543


In [25]:
import cv2
import easyocr
import numpy as np

# Carregar os nomes das classes
with open("classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Carregar o modelo YOLO pré-treinado e as classes
net = cv2.dnn.readNet("custom-yolov4-detector_final.weights", "custom-yolov4-detector.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Carregar a imagem
image = cv2.imread('6.jpg')
height, width, channels = image.shape

# Preparar a imagem para YOLO
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Processar as detecções
class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Ajuste o limiar de confiança conforme necessário
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Aplicar Non-Max Suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Cortar a região da placa
for i in range(len(boxes)):
    if i in indices:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        print(label)
        if label == "placa":  # Certifique-se de que a classe está correta
            roi = image[y:y+h, x:x+w]
            # Salvar a região da placa
            cv2.imwrite('placa.jpg', roi)

# Ler a região da placa usando EasyOCR
reader = easyocr.Reader(['pt'])
results = reader.readtext('placa.jpg')

# Imprimir os resultados
for (bbox, text, prob) in results:
    print(f'Texto: {text}, Confiança: {prob}')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


placa
Texto: lesl, Confiança: 0.0570419542491436
Texto: Pro8Foo, Confiança: 0.4702001546750024


In [1]:
import os
# Obter o caminho absoluto do arquivo de credenciais na pasta atual do projeto
current_directory = os.getcwd()
credential_path = os.path.join(current_directory, "application_default_credentials.json")

# Definir a variável de ambiente GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path


In [ ]:
import os
from google.cloud import vision
import re


# Configura o cliente da API
client = vision.ImageAnnotatorClient()

def extract_license_plate(image_path):
    try:
        with open(image_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        texts = response.text_annotations

        if not texts:
            return "Nenhum texto detectado na imagem."

        full_text = texts[0].description
        print(f"Texto detectado: {full_text}")

        plate_pattern = r'[A-Z]{3}[0-9][A-Z0-9][0-9]{2}'
        matches = re.findall(plate_pattern, full_text)

        if matches:
            return matches
        else:
            return "Nenhuma placa encontrada no texto detectado."
    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")
        return None

# Exemplo de uso
image_path = "8.PNG"  # Substitua pelo caminho da sua imagem
plates = extract_license_plate(image_path)
print(f"Placas encontradas: {plates}")


Texto detectado: ◄ Busca.
21:36
HABILITAÇÃO
Atualizado em: 10/12/2024 - 21:30:10
Verifique autenticidade do QR Code com o app Vio
VÁLIDA EM TODO
O TERRITÓRIO NACIONAL
2259590237
REPÚBLICA FEDERATIVA DO BRASIL
MINISTÉRIO DA INFRAESTRUTURA
DEPARTAMENTO NACIONAL DE TRÂNSITO
CARTEIRA NACIONAL DE HABILITAÇÃO
NOME
ISAC SOARES CAMARA JUNIOR
DOC. IDENTIDADE/ORG EMISSOR/UF
2617795 SSP DF
CPF
016.016.291-25
-FILIAÇÃO
ISAC SOARES CAMARA
DATA NASCIMENTO-
19/09/1987
N° REGISTRO
03931237097
VALIDADE
22/06/2031
B
CAT. HAB.-
1 HABILITAÇÃO
16/09/2006
MIRIAN ANDRELINO SOARES CAM
ARA
PERMISSÃO
ACC
D
OF
Histórico de emissões da CNH
Placas encontradas: Nenhuma placa encontrada no texto detectado.


In [10]:
#listando arquivos do diretorio
from pathlib import Path

diretorio = Path("imagens/documentos")

# Listar todos os arquivos
for arquivo in diretorio.iterdir():
    if arquivo.is_file():
        print(arquivo)

imagens\documentos\1.jpg
imagens\documentos\2.jpg
imagens\documentos\3.jpg


In [9]:
#extraindo metadados
import exifread

def get_image_datetime(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

        # A data e hora geralmente estão em tags como 'DateTimeOriginal' ou 'DateTimeDigitized'
        datetime_original = tags.get('EXIF DateTimeOriginal')
        datetime_digitized = tags.get('EXIF DateTimeDigitized')

        if datetime_original:
            return datetime_original.values
        elif datetime_digitized:
            return datetime_digitized.values
        else:
            return "Data não encontrada nos metadados EXIF"

# Substitua 'caminho/para/sua/imagem.jpg' pelo caminho da sua imagem
image_path = 'imagens/documentos/1.jpg'
datetime = get_image_datetime(image_path)
print(datetime)

2024:12:09 22:01:55


In [15]:
#listando arquivos do diretorio
from pathlib import Path
def list_files():
    dir_docs = Path("imagens/documentos")
    dir_plates = Path("imagens/placas")
    docs_paths=[file for file in dir_docs.iterdir() if file.is_file()]
    plates_paths = [file for file in dir_plates.iterdir() if file.is_file()]
    return docs_paths,plates_paths


In [17]:
docs_paths, plates_paths = list_files()

In [19]:
get_image_datetime(docs_paths[0])

'2024:12:09 22:01:55'

In [26]:
docs_dates = list(map(get_image_datetime,docs_paths))

In [27]:
docs_dates

['2024:12:09 22:01:55', '2024:12:10 22:01:01', '2024:12:11 22:37:48']

In [36]:
list(zip(docs_paths,docs_dates))

[(WindowsPath('imagens/documentos/1.jpg'), '2024:12:09 22:01:55'),
 (WindowsPath('imagens/documentos/2.jpg'), '2024:12:10 22:01:01'),
 (WindowsPath('imagens/documentos/3.jpg'), '2024:12:11 22:37:48')]

In [35]:
docs_paths_dates = list(zip(docs_paths,docs_dates))
#Organizando por data
from datetime import datetime

# Função para converter a string de data em um objeto datetime
def parse_datetime(date_str):
    return datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')

# Organizar a lista pela data
sorted_docs_dates = sorted(docs_paths_dates, key=lambda x: parse_datetime(x[1]))

# Exibir a lista organizada
for doc in sorted_docs_dates:
    print(doc)


(WindowsPath('imagens/documentos/1.jpg'), '2024:12:09 22:01:55')
(WindowsPath('imagens/documentos/2.jpg'), '2024:12:10 22:01:01')
(WindowsPath('imagens/documentos/3.jpg'), '2024:12:11 22:37:48')


In [37]:
def sort_by_date(files_path):
    files_date = list(map(get_image_datetime,files_path))
    files_path_date = list(zip(files_path,files_date))
    sorted_files_date = sorted(files_path_date, key=lambda x: parse_datetime(x[1]))
    return sorted_files_date

In [40]:
sorted_plates_dates = sort_by_date(plates_paths)

In [41]:
sorted_plates_dates

[(WindowsPath('imagens/placas/5.jpg'), '2024:12:09 23:04:10'),
 (WindowsPath('imagens/placas/6.jpg'), '2024:12:10 23:04:27'),
 (WindowsPath('imagens/placas/7.jpg'), '2024:12:11 23:04:42')]

In [42]:
docs_img_list = []
plates_img_list = []
for path in sorted_docs_dates:
    try:
        image_path = path[0]
        with open(image_path, 'rb') as image_file:
                content = image_file.read()  
    except:
        print('Deu ruim')
    docs_img_list.append(content)
for path in sorted_plates_dates:
    try:
        image_path = path[0]
        with open(image_path, 'rb') as image_file:
                content = image_file.read()  
    except:
        print('Deu ruim')
    plates_img_list.append(content)

In [49]:
imgs_list =  [item for pair in zip(docs_img_list, plates_img_list) for item in pair]

In [75]:
import base64
with open('imagens/documentos/1.jpg', 'rb') as f:
    img1 = base64.b64encode(f.read()).decode('utf-8')

with open('imagens/placas/5.jpg', 'rb') as f:
    img2 = base64.b64encode(f.read()).decode('utf-8')

In [ ]:
import io
from pathlib import Path #ler os diretorios e listar arquivos
import exifread #Para ler os metadados de data
from datetime import datetime
import base64 #converter pra o que o vertex entende
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting
import pandas as pd
from io import StringIO

#lista arquivos do diretorios documentos e placas
def list_files(verbose=1):
    dir_docs = Path("imagens/documentos")
    dir_plates = Path("imagens/placas")
    docs_paths=[file for file in dir_docs.iterdir() if file.is_file()]
    plates_paths = [file for file in dir_plates.iterdir() if file.is_file()]
    if verbose:
        print(f'Encontrados {len(docs_paths)} na pasta documentos')
        print(f'Encontrado {len(plates_paths)} na pasta de placas')
    return docs_paths,plates_paths

#extraindo metadados
# Recebe um caminho para arquivo de imagem e pega a data do EXIF
def get_image_datetime(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

        # A data e hora geralmente estão em tags como 'DateTimeOriginal' ou 'DateTimeDigitized'
        datetime_original = tags.get('EXIF DateTimeOriginal')
        datetime_digitized = tags.get('EXIF DateTimeDigitized')

        if datetime_original:
            return datetime_original.values
        elif datetime_digitized:
            return datetime_digitized.values
        else:
            return "Data não encontrada nos metadados EXIF"
        


# Função para converter a string de data em um objeto datetime
def parse_datetime(date_str):
    return datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')


#Organiza por data  
#Recebe uma lista de caminhos para arquivos, paga a data do EXIF de cada com a função get_image_datetime
#Organiza em uma tupla caminho do arquivo, data da foto.
# Organiza por data 
# Retorna a lista organizada por data 
def sort_by_date(files_path, verbose=1):
    files_date = list(map(get_image_datetime,files_path))
    files_path_date = list(zip(files_path,files_date))
    sorted_files_date = sorted(files_path_date, key=lambda x: parse_datetime(x[1]))
    if verbose:
        print(f'Organizado {len(sorted_files_date)} arquivos por data')
    return sorted_files_date

def img_list_loader(list_of_img_path, verbose=1):
    img_list = []
    if verbose:
        print(f'Carregando {len(list_of_img_path)} imagens')
    for path in list_of_img_path:
        try:
            image_path = path[0]
            with open(image_path, 'rb') as image_file:
                    content = image_file.read()  
        except:
            print('Deu ruim')
        img_list.append(content)
    if verbose:
        print(f'Carregado {len(img_list)} para memória')
    return img_list

#Converte para o formato que o vertexAI entende
def vertex_img_converter(img):
    encoded_img = base64.b64encode(img).decode('utf-8')
    vertex_img = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(encoded_img))
    return vertex_img

def convert_to_excel(output, file_name="output"):
    # Converter a saída para um DataFrame do pandas 
    output_io = StringIO(output)
    df = pd.read_table(output_io, sep="|", skiprows=0, skipinitialspace=True).reindex()
    df.dropna(axis=1, how="all", inplace=True)
    df.drop(index=0, inplace=True)
    # Salvar como arquivo Excel 
    df.to_excel(f"{file_name}.xlsx", index=False, engine="openpyxl") 


#Gera tabela com nome do condutor,rg,cpf, placa, modelo e marca do carro.
def generate(verbose=1):

    #lista os arquivos nas pastas de documentos e de fotos das placas
    docs_paths, plates_paths = list_files(verbose=verbose) 
    #pega as datas das fotos e organiza pelas datas
    sorted_docs_dates, sorted_plates_dates = sort_by_date(docs_paths,verbose=verbose), sort_by_date(plates_paths,verbose=verbose)
    #carrega as imagens e coloca em um lista
    docs_img_list, plates_img_list = img_list_loader(sorted_docs_dates,verbose=verbose), img_list_loader(sorted_plates_dates, verbose=verbose)
    #organiza em um lista só primeiro documento depois placa
    if verbose: print('Organizando documentos e placas, um a um')
    imgs_list =  [item for pair in zip(docs_img_list, plates_img_list) for item in pair]
    #converte para o formato entendido pelo vertex
    if verbose: print('Convertendo imagens para formato da API Google Cloud Vertex AI')
    vertex_imgs_list = list(map(vertex_img_converter,imgs_list))
    if verbose: print('Iniciando chamada para API')    

    vertexai.init(project="guardautomation", location="southamerica-east1")
    model = GenerativeModel(
        "gemini-1.5-pro-002",
    )
    # Combina as imagens com o prompt
    prompt = vertex_imgs_list + [text1]
    responses = model.generate_content(
        prompt,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    # Usar StringIO para capturar a saída do loop de impressão 
    output_buffer = io.StringIO()
    if verbose: print('Recebendo resposta', end='') 
    for response in responses:
        if verbose: print('.',end='') 
        output_buffer.write(response.text) 
    
    # Salvar a saída na variável output 
    output = output_buffer.getvalue() 
    output_buffer.close() 
    if verbose:
        print('')
        print('Resposta recebida')
    return output

text1 = """Crie uma tabela com a placa, modelo, marca do carro, cor,  nome, cpf, rg. 
Nome, cpf e rg estão nas imagens posteriores a dos carros, 
a imagem 1 com a 2 e a 3 com a 4 e assim posteriormente.
rg é geralmente identificado por doc. identidade e seguido pelo orgão emissor, cpf tem 11 digitos.
Se for possivel retirar algum dados de alguma foto marque a informação com ilegível. 
Responda somente a tabela nada além da tabela, sem texto antes ou depois da tabela"""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

output = generate(verbose=False)
#convert_to_excel(output)
#print(output)


Organizado 23 arquivos por data
Organizado 23 arquivos por data


In [5]:
from google.cloud import storage
import uuid
import os

def generate_uuid():
   uuid_str = str(uuid.uuid4())
   return uuid_str

def upload_imgs(imgs,img_type, uuid_str):
    """Uploads imagens para o Cloud Storage usando um UUID como prefixo."""

    storage_client = storage.Client()
    bucket_name = "guardhouse_automation_bucket"
    bucket = storage_client.bucket(bucket_name)
    remote_files = []
    for img in imgs:
      file_name = os.path.basename(img)
      remote_name = f"{uuid_str}/{img_type}/{file_name}" # nome do arquivo com o prefixo UUID
      blob = bucket.blob(remote_name)
      blob.upload_from_filename(img)
      url_img = blob.public_url
      remote_files.append(url_img)
      print(f"Imagem {file_name} enviada para {url_img}.")

    return remote_files # retorna uma lista de links publicos e o UUID

def deletar_pastas(uuid_str):
  storage_client = storage.Client()
  bucket_name = "guardhouse_automation_bucket"
  bucket = storage_client.bucket(bucket_name)
  blobs = bucket.list_blobs(prefix=f"{uuid_str}/") #lista todos arquivos com prefixo
  for blob in blobs:
    blob.delete()
  print(f"Pasta com UUID {uuid_str} deletada.")


In [10]:
# Exemplo de uso
docs_paths, plates_paths = list_files(verbose=False)
uuid_gerado = generate_uuid()
links_documentos = upload_imgs(docs_paths,'documentos', uuid_gerado)
links_placas = upload_imgs(plates_paths,'placas', uuid_gerado)
# ... (Processar as imagens com a API do Vertex AI) ...
print("UUID para rastreamento: ",uuid_gerado)
#deletar_pastas(uuid_gerado)
print(links_documentos)
print(links_placas)

Imagem 1.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/1.jpg.
Imagem 10.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/10.jpg.
Imagem 11.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/11.jpg.
Imagem 13.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/13.jpg.
Imagem 14.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/14.jpg.
Imagem 15.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/15.jpg.
Imagem 16.jpg enviada para https://storage.googleapis.com/guardhouse_automation_bucket/9eb3c337-bccf-400b-920e-60c7ff2a42b4/documentos/16.jpg.
I